**Ronaldlee Ejalu**

**CSC 583**

**HW#4**

In [153]:
import csv
import re
import os
import tarfile
import numpy as np
import pandas as pd
import pickle
import random
import nltk
nltk.download("punkt")
# tokenize the string without whitespaces, newline and tabs
from nltk.tokenize import WhitespaceTokenizer 
from nltk.util import ngrams
import itertools
import collections
from collections import Counter
pd.options.display.max_columns = None

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [154]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [155]:
%cd /gdrive/MyDrive/CSC583

/gdrive/MyDrive/CSC583


neg and pos diirectories' path

In [156]:
directoryPath_neg = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/neg'
directoryPath_pos = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/pos'


train and test directories' path

In [157]:
directoryPath_train_o = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/train'
directoryPath_test_o = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/test'

In [158]:
directoryPath_train_labels = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/train_labels'
directoryPath_test_labels = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/test_labels'

regular expressions for extracting train and test sets

In [159]:
wordPattern_pos = r'^cv[0-7]{1}'
wordPattern_neg = r'^cv[8-9]{1}'

helper functions

In [160]:
#train_groundTruth_L = []
#test_groundTruth_L = []
def deriveTrainAndTest(directory_for_pos_or_neg, wordPattern_pos, srcPathFiles, destFilePath, labels, groudTruth_L):
  """Function that uses a regular expression that assigns files that start with cv0 
  through cv7 to the training and files that start with cv8 and cv9 to the test set"""
  
  for file in directory_for_pos_or_neg:
    t = re.findall(wordPattern_pos, file)
    if len(t) > 0:
      srcPath = os.path.join(srcPathFiles, file)
      destPath = os.path.join(destFilePath, file)
      os.rename(srcPath, destPath)
      groudTruth_L.append(labels)         

In [161]:
def readPlainDoc(filename_s):
  """Function that returns a string in lower case after reading the contents of the file"""
  fileNamePath = "data/homework3/sentiment_lexicon/" + filename_s
  with open(fileNamePath, 'r',  encoding = 'ISO-8859-1') as sampleFile:
    fileContents = sampleFile.read()
  # normalize all words to lower case
  return fileContents.lower()

In [162]:
def dumpListToFile(listName, groundTruth_labels):
  fileName = '/gdrive/MyDrive/CSC583/data/homework3/data/' + groundTruth_labels + '.csv'
  with open(fileName, 'w') as file:
    for item in listName:
      file.write(str(item) +'\n')

In [163]:
def readCSVFiles(fileName):
  filePath = '/gdrive/MyDrive/CSC583/data/homework3/data/' + fileName
  fileItems = []
  with open(filePath, 'r') as file:
    csvFile = csv.reader(file)
    for line in csvFile:
      fileItems.append(line)
  return fileItems

In [164]:
def cntPosNeg_adjec(tokenizedText, lexicon_positive_words, cnt_pos_adjec):
  """Function that returns either the number of positive or negative adjectives
  """
  pos_text = nltk.pos_tag(tokenizedText)
  for p_item in pos_text:
    if p_item[1].startswith('J') and p_item[0] in lexicon_positive_words:
      cnt_pos_adjec += 1
  return cnt_pos_adjec

In [165]:
def readDirectoryFiles(directoryFiles_train, filepath):
  """Function that returns a list of documents after reading all files in the directory"""
  train_L_Files = directoryFiles_train
  train_text_L = []
  
  for file in train_L_Files:
    filePath = filepath + file
    f_c = open(filePath, "r")
    train_text_L.append(f_c.read())
  return train_text_L

In [166]:
df_reviews = pd.DataFrame()
def remove_blanks(pd_f = df_reviews):
  # remove blanks from review
  blanks = []  # start with an empty list
  
  for lb,rv in pd_f.itertuples():
    if type(rv)==str:
      if rv.isspace():
        blanks.append(lb)     
  pd_f.drop(blanks, inplace=True)
  return pd_f

In [167]:
reviews_train_df=pd.DataFrame()
def segmentDataIntoPosOrNeg(pd_f=reviews_train_df, no_flag=1):
  """Function that returns either positive or negative reviews data frame"""
  filter_neg_t = (pd_f['labels'] == no_flag)
  df_train_reviews_neg = pd_f[filter_neg_t]
  return df_train_reviews_neg # return a data frame

In [168]:
df_train_reviews_pos_0_1=pd.DataFrame()
def process_reviews_concatenated_sent(pd_f=df_train_reviews_pos_0_1):
  """Function that returns a chained list of all sentences combining all reviews together"""
  reviews_pos_L = pd_f['reviews'].values.tolist() # convert the data frame column into a list
  left_starter_marker = '<s>'
  right_end_marker = '</s>'
  review_L = []
  wtk = WhitespaceTokenizer()
  for review  in reviews_pos_L:
    # break down the review into sentences
    sentences = nltk.sent_tokenize(review)
    list_of_tk_sent = []
    for item in range(len(sentences)):
      # for each sentence append the right and left markers
      s = left_starter_marker + ' ' + sentences[item] + ' ' + right_end_marker
      s = wtk.tokenize(s)
      
      # add the tokenized sentence to a list 
      list_of_tk_sent.append(s)
    #print('printing the list before chaining: %s' %str(list_of_tk_sent))
      # derive the chained list of sentences of a review 
    chain_L = list(itertools.chain.from_iterable(list_of_tk_sent))
    # add the chained list of sentences of a review to the list
    review_L.append(chain_L)
    #print('lenght of the list: %s' %str(len(review_L)))
  # chain the lists of the different reviews' sentences together
  chained_reviews_L = list(itertools.chain.from_iterable(review_L)) 
  #print('length of chained list: %s' %str(len(chained_reviews_L)))
  
  return chained_reviews_L

In [169]:
dict_pos = {}
def view_sample_items_bi_grams(dict_pos):
  """Function that iterates over a 
  bigram while printing its contents
  """
  count = 1
  for x, y in dict_pos:
    if count <= 5:
      print('[2 - {}] <{}, {}>'.format(count, x, y))
    else:
      break
    count +=1 

In [170]:
tri_grams_reviews_pos_train = zip()
def view_sample_items_tri_grams(tri_grams_reviews_pos_train):
  """Function that iterates over a trigram as it prints its contents"""
  tri_grams_reviews_pos_train_c = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in tri_grams_reviews_pos_train]
  count = 0
  for item in tri_grams_reviews_pos_train_c:
    if count <= 5:
      print('[3 - {}] {}'.format(count, item))
    else:
      break
    count += 1


In [171]:
quad_grams_reviews_pos_train = zip()
def view_sample_items_quad_grams(quad_grams_reviews_pos_train):
  """Function that iterates over a quadgram as it prints its contents"""
  quad_grams_reviews_pos_train_c = [((x_0, x_1, x_2), x_3)  for (x_0, x_1, x_2, x_3) in quad_grams_reviews_pos_train]
  count = 0
  for item in quad_grams_reviews_pos_train_c:
    if count <= 5:
      print('[4 - {}] {}'.format(count, item))
    else:
      break
    count += 1


In [172]:
def deriveBigram_model(cfd_bi_gram_train):
  """Function that return a dictionary of probabilities of the next word given the history"""
  bigram_modal = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
  for key, val in cfd_bi_gram_train.items():
    # collect all bigram frequencies
    for k, v in val.items():
      bigram_modal[key][k] = v

  for x1 in bigram_modal:
    bi_modal_cnt = float(sum(bigram_modal[x1].values()))
    #print(x1, bi_modal_cnt)
    # derive the probabilities of the bigram
    for x2 in bigram_modal[x1]:
      bigram_modal[x1][x2]  = bigram_modal[x1][x2] / bi_modal_cnt
      #print(x1, x2, bigram_modal[x1][x2])

  return bigram_modal

In [173]:
# computing perplexity with respect to the positive or negative test set 
def bigram_probabilities_testSet(cfd_bi_gram_train_pos, bi_grams_reviews_pos_test):
  probabilities = []
  # build a bigram model
  bigram_modal = deriveBigram_model(cfd_bi_gram_train_pos)
  for key, val in bi_grams_reviews_pos_test:
    value = bigram_modal[key].get(val, 0)
    if value == 0:
      probabilities.append(0.00)
    else:
      probabilities.append(np.log(value))
    #print(key, val, value)
    # get the log of the probablilities
  lnSum=np.sum(probabilities)
  return lnSum

In [174]:
reviews_pos_test=[]
cfd_bi_gram_train_pos=nltk.ConditionalFreqDist()
bi_grams_reviews_pos_test = []

def computePerplexity(list_L=reviews_pos_test, ngramType='Pos-Pos', cfd_type=cfd_bi_gram_train_pos,\
                      dataset_l = bi_grams_reviews_pos_test):
  """Compute the perplexity for with respect to the Pos and negative test data"""
  n = len(list_L)
  ln_sum=bigram_probabilities_testSet(cfd_type, dataset_l)
  print('ln_sum for the {}: {}'.format(ngramType,ln_sum))
  ans=np.exp(-ln_sum/n)
  perplexity[ngramType] = ans

In [175]:
def deriveTrigram_modal(cfd_tri_gram_train_pos):
  trigram_modal = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
  for key, val in cfd_tri_gram_train_pos.items():
    # collect all trigram frequencies
    for k, v in val.items():
      #print(key, k, v)
      trigram_modal[key][k] = v
  # populate the modal with probabilities
  for x_0_x_1 in trigram_modal:
    trigram_modal_cnt = float(sum(trigram_modal[x_0_x_1].values()))
    #print(x_0_x_1, trigram_modal_cnt)
    # derive the probabilities of the bigram 
    for x_2 in trigram_modal[x_0_x_1]:
      trigram_modal[x_0_x_1][x_2]  = trigram_modal[x_0_x_1][x_2] / trigram_modal_cnt
      #print(x_0_x_1, x_2, trigram_modal[x_0_x_1][x_2])
  return trigram_modal

In [176]:
def trigram_probabilities_testSet(cfd_tri_gram_train_pos, tri_grams_reviews_neg_test_L):
  """Function that return sum of the probabilities of the ngram model"""
  probabilities = []
  trigram_modal = deriveTrigram_modal(cfd_tri_gram_train_pos) # build the trigram modal
  for key, val in tri_grams_reviews_neg_test_L:
    value = trigram_modal[key].get(val, 0)
    if value == 0:
      probabilities.append(0.00)
    else:
      probabilities.append(np.log(value))
    #print(key, val, value)
  lnSum=np.sum(probabilities)
  return lnSum


In [177]:
def deriveQuadgram_modal(cfd_quad_gram_train_pos):
  """Function that returns the quadgram model with probabilities """
  quadgram_modal = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
  for key, val in cfd_quad_gram_train_pos.items():
    # collect all quadgram frequencies
    for k, v in val.items():
      #print(key, k, v)
      quadgram_modal[key][k] = v
  # populate the modal with probabilities
  for x_0_x_1_x_2 in quadgram_modal:
    quadgram_modal_cnt = float(sum(quadgram_modal[x_0_x_1_x_2].values()))
    # derive the probabilities of the quadgram
    for x_3 in quadgram_modal[x_0_x_1_x_2]:
      quadgram_modal[x_0_x_1_x_2][x_3]  = quadgram_modal[x_0_x_1_x_2][x_3] / quadgram_modal_cnt
      #print(x_0_x_1_x_2, x_3, quadgram_modal[x_0_x_1_x_2][x_3])
  return quadgram_modal

In [178]:
reviews_neg_test=[]
tri_grams_reviews_neg_test_L=[]
cfd_tri_gram_train_pos = nltk.ConditionalFreqDist()
def computeTrigramPerplexity(list_L=reviews_neg_test, ngramType='Pos-Neg-trigram', cfd_type=cfd_tri_gram_train_pos,\
                      dataset_l = tri_grams_reviews_neg_test_L):
  """Compute the perplexity for with respect to the Pos and negative test data"""
  n = len(list_L)
  ln_sum=trigram_probabilities_testSet(cfd_type, dataset_l)
  print('ln_sum for the {}: {}'.format(ngramType,ln_sum))
  ans=np.exp(-ln_sum/n)
  perplexity[ngramType] = ans

In [179]:
def quadgram_probabilities_testSet(cfd_quad_gram_train_pos, quad_grams_reviews_pos_test_L):
  """Function that return sum of the probabilities of the ngram model"""
  probabilities = []
  quadgram_modal = deriveQuadgram_modal(cfd_quad_gram_train_pos) # build the quadgram modal
  for key, val in quad_grams_reviews_pos_test_L:
    value = quadgram_modal[key].get(val, 0)
    if value == 0:
      probabilities.append(0.00)
    else:
      probabilities.append(np.log(value))
    #print(key, val, value)
  lnSum=np.sum(probabilities)
  return lnSum

In [180]:
reviews_pos_test=[]
quad_grams_reviews_pos_test_L=[]
cfd_quad_gram_train_pos = nltk.ConditionalFreqDist()
def computequadgramPerplexity(list_L=reviews_pos_test, ngramType='Pos-Pos-quadgram', cfd_type=cfd_quad_gram_train_pos,\
                      dataset_l = quad_grams_reviews_pos_test_L):
  """Compute the perplexity for with respect to the Pos and Pos test data"""
  n = len(list_L)
  ln_sum=quadgram_probabilities_testSet(cfd_quad_gram_train_pos, quad_grams_reviews_pos_test_L)
  print('ln_sum for the {}: {}'.format(ngramType,ln_sum))
  ans=np.exp(-ln_sum/n)
  perplexity[ngramType] = ans

In [181]:
#uncomment this line to download the file
!wget -c http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz -P data/homework3

--2023-05-05 02:17:36--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘data/homework3/review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  1.86MB/s    in 1.6s    

2023-05-05 02:17:39 (1.86 MB/s) - ‘data/homework3/review_polarity.tar.gz’ saved [3127238/3127238]



In [182]:
# uncomment this code to extract the file
#open the file
file = tarfile.open("data/homework3/review_polarity.tar.gz")

# extract the file
file.extractall('data/homework3')
file.close()

In [183]:
# uncomment the lines below to create directories
%rm -rf data/homework3/txt_sentoken/train
%rm -rf data/homework3/txt_sentoken/test
%mkdir data/homework3/txt_sentoken/train
%mkdir data/homework3/txt_sentoken/test

In [184]:
len(os.listdir(directoryPath_test_o))

0

In [185]:
len(os.listdir(directoryPath_train_o))

0

In [186]:
directoryFiles_pos = os.listdir(directoryPath_pos)
directoryFiles_neg = os.listdir(directoryPath_neg)
print("The number of files in the pos directory are %d" %len(directoryFiles_pos))
print("The number of files in the neg directory are %d" %len(directoryFiles_neg))

The number of files in the pos directory are 1000
The number of files in the neg directory are 1000


In [187]:
# initialize empty lists 
train_groundTruth_L = []
test_groundTruth_L = []

In [188]:
deriveTrainAndTest(directoryFiles_pos, wordPattern_pos,directoryPath_pos,directoryPath_train_o, 1, train_groundTruth_L)
deriveTrainAndTest(directoryFiles_neg, wordPattern_pos, directoryPath_neg, directoryPath_train_o, 0, train_groundTruth_L)
deriveTrainAndTest(directoryFiles_pos, wordPattern_neg, directoryPath_pos, directoryPath_test_o, 1, test_groundTruth_L)
deriveTrainAndTest(directoryFiles_neg, wordPattern_neg, directoryPath_neg, directoryPath_test_o, 0, test_groundTruth_L)

In [189]:
len(train_groundTruth_L)

1600

In [190]:
len(test_groundTruth_L)

400

In [191]:
# write the groundTruth_training list to the file
dumpListToFile(train_groundTruth_L, 'train_groundTruth_L')

In [192]:
# write the groundTruth_test list to the file
dumpListToFile(test_groundTruth_L, 'test_groundTruth_L')

In [193]:
directoryFiles_train = os.listdir(directoryPath_train_o)
directoryFiles_test = os.listdir(directoryPath_test_o)
print("There are %d files in the train data set." %len(directoryFiles_train))
print("There are %d files in the test data set." %len(directoryFiles_test))


There are 1600 files in the train data set.
There are 400 files in the test data set.


Get a list of files from the training data set

In [194]:
train_filepath='data/homework3/txt_sentoken/train/'
train_L_Files  = readDirectoryFiles(directoryFiles_train, train_filepath)
print("%d files of reviews read from the training data set." %len(train_L_Files))

test_filepath='data/homework3/txt_sentoken/test/'
test_L_Files  = readDirectoryFiles(directoryFiles_test, test_filepath)
print("%d files of reviews read from the test data set." %len(test_L_Files))


1600 files of reviews read from the training data set.
400 files of reviews read from the test data set.


# Derive the training data frame with all its reviews and labels

In [195]:
reviews_train_df = pd.DataFrame(data=list(zip(train_L_Files, train_groundTruth_L)), columns=['reviews', 'labels'])

In [196]:
reviews_train_df.tail()

,reviews,labels
1595,"the central focus of michael winterbottom's "" ...",0
1596,"after the average mouse hunt , the silly small...",0
1597,i have seen several ( but not that many ) wood...,0
1598,fritz lang's first american film since leaving...,0
1599,veteran actor clint eastwood has never looked ...,0


In [197]:
reviews_train_df.groupby(['labels'])['labels'].count()

labels
0    800
1    800
Name: labels, dtype: int64

# Derive the test data frame with all its reviews and labels

In [198]:
reviews_test_df = pd.DataFrame(data=list(zip(test_L_Files, test_groundTruth_L)), columns=['reviews', 'labels'])

In [199]:
reviews_test_df.head()

,reviews,labels
0,salaries of hollywood top actors are getting o...,1
1,"movies like six days , seven nights make me ma...",1
2,"if anything , "" stigmata "" should be taken as ...",1
3,"john boorman's "" zardoz "" is a goofy cinematic...",1
4,the kids in the hall are an acquired taste . \...,1


In [200]:
reviews_test_df.groupby(['labels'])['labels'].count()

labels
0    200
1    200
Name: labels, dtype: int64

## Separate the training reviews into positive and negative

In [201]:
# positive training reviews
df_train_reviews_pos = segmentDataIntoPosOrNeg(reviews_train_df, 1)
df_train_reviews_pos.groupby(['labels'])['labels'].count()

labels
1    800
Name: labels, dtype: int64

In [202]:
# negative training reviews
df_train_reviews_neg = segmentDataIntoPosOrNeg(reviews_train_df, 0)
df_train_reviews_neg.groupby(['labels'])['labels'].count()

labels
0    800
Name: labels, dtype: int64

## Separate the test reviews into positive and negative.

In [203]:
# positive test reviews
df_test_reviews_pos = segmentDataIntoPosOrNeg(reviews_test_df, 1)
df_test_reviews_pos.groupby(['labels'])['labels'].count()

labels
1    200
Name: labels, dtype: int64

In [204]:
# negative test reviews
df_test_reviews_neg = segmentDataIntoPosOrNeg(reviews_test_df, 0)
df_test_reviews_neg.groupby(['labels'])['labels'].count()

labels
0    200
Name: labels, dtype: int64

# Derive long lists of concatenated sentences

In [205]:
# positive train reviews
df_train_reviews_pos.shape

(800, 2)

In [206]:
# invoke a function that returns a chained list of all sentences after combining all reviews together
reviews_pos_train = process_reviews_concatenated_sent(df_train_reviews_pos)

In [207]:
# negative train reviews
df_train_reviews_neg.shape

(800, 2)

In [208]:
# invoke a function that returns a chained list of all sentences after combining all reviews together
reviews_neg_train = process_reviews_concatenated_sent(df_train_reviews_neg)

In [209]:
# postive test reviews
df_test_reviews_pos.shape

(200, 2)

In [210]:
# invoke a function that returns a chained list of all sentences after combining all reviews together
reviews_pos_test = process_reviews_concatenated_sent(df_test_reviews_pos)

In [211]:
# negative test reviews
df_test_reviews_neg.shape

(200, 2)

In [212]:
# invoke a function that returns a chained list of all sentences after combining all reviews together
reviews_neg_test = process_reviews_concatenated_sent(df_test_reviews_neg)

# Build bigram, trigram and quadgram models.

## training sets

###bigrams

In [213]:
# creating positive training bigrams using nltk.util.ngrams
bi_grams_reviews_pos_train = ngrams(reviews_pos_train, 2)

In [214]:
# look at the sample contents of the positive training bi gram
view_sample_items_bi_grams(bi_grams_reviews_pos_train)

[2 - 1] <<s>, disaster>
[2 - 2] <disaster, films>
[2 - 3] <films, have>
[2 - 4] <have, a>
[2 - 5] <a, tendency>


In [215]:
# creating negative training bigrams using nltk.util.ngrams
bi_grams_reviews_neg_train = ngrams(reviews_neg_train, 2)

In [216]:
# look at the sample contents of the negative training bi gram
view_sample_items_bi_grams(bi_grams_reviews_neg_train)

[2 - 1] <<s>, it>
[2 - 2] <it, used>
[2 - 3] <used, to>
[2 - 4] <to, be>
[2 - 5] <be, that>


### trigrams

In [217]:
# create the positive training trigram
tri_grams_reviews_pos_train = ngrams(reviews_pos_train, 3)

In [218]:
# look at the sample contents of the positive training trigram
view_sample_items_tri_grams(tri_grams_reviews_pos_train)

[3 - 0] (('<s>', 'disaster'), 'films')
[3 - 1] (('disaster', 'films'), 'have')
[3 - 2] (('films', 'have'), 'a')
[3 - 3] (('have', 'a'), 'tendency')
[3 - 4] (('a', 'tendency'), 'to')
[3 - 5] (('tendency', 'to'), 'be')


In [219]:
# create the negative training trigram 
tri_grams_reviews_neg_train = ngrams(reviews_neg_train, 3)

In [220]:
# look at the sample contents of the negative training trigram
view_sample_items_tri_grams(tri_grams_reviews_neg_train)

[3 - 0] (('<s>', 'it'), 'used')
[3 - 1] (('it', 'used'), 'to')
[3 - 2] (('used', 'to'), 'be')
[3 - 3] (('to', 'be'), 'that')
[3 - 4] (('be', 'that'), 'not')
[3 - 5] (('that', 'not'), 'just')


###quadgrams

In [221]:
# create the positive training quadgram
quad_grams_reviews_pos_train = ngrams(reviews_pos_train, 4)

In [222]:
# look at the sample contents of the positive training quadgram
view_sample_items_quad_grams(quad_grams_reviews_pos_train)

[4 - 0] (('<s>', 'disaster', 'films'), 'have')
[4 - 1] (('disaster', 'films', 'have'), 'a')
[4 - 2] (('films', 'have', 'a'), 'tendency')
[4 - 3] (('have', 'a', 'tendency'), 'to')
[4 - 4] (('a', 'tendency', 'to'), 'be')
[4 - 5] (('tendency', 'to', 'be'), 'very')


In [223]:
# create the negative training quadgram
quad_grams_reviews_neg_train = ngrams(reviews_neg_train, 4)

In [224]:
# look at the sample contents of the negative training quadgram
view_sample_items_quad_grams(quad_grams_reviews_neg_train)

[4 - 0] (('<s>', 'it', 'used'), 'to')
[4 - 1] (('it', 'used', 'to'), 'be')
[4 - 2] (('used', 'to', 'be'), 'that')
[4 - 3] (('to', 'be', 'that'), 'not')
[4 - 4] (('be', 'that', 'not'), 'just')
[4 - 5] (('that', 'not', 'just'), 'anyone')


## testing sets

###bigrams

In [225]:
# creating positive training bigrams using nltk.util.ngrams
bi_grams_reviews_pos_test = ngrams(reviews_pos_test, 2)

In [226]:
# look at the sample contents of the positive test bigram
view_sample_items_bi_grams(bi_grams_reviews_pos_test)

[2 - 1] <<s>, salaries>
[2 - 2] <salaries, of>
[2 - 3] <of, hollywood>
[2 - 4] <hollywood, top>
[2 - 5] <top, actors>


In [227]:
# creating negative test bigrams
bi_grams_reviews_neg_test = ngrams(reviews_neg_test, 2)

In [228]:
# look at the sample contents of the negative test bigram
view_sample_items_bi_grams(bi_grams_reviews_neg_test)

[2 - 1] <<s>, those>
[2 - 2] <those, of>
[2 - 3] <of, you>
[2 - 4] <you, who>
[2 - 5] <who, frequently>


###trigrams

In [229]:
# create the positive training trigram
tri_grams_reviews_pos_test = ngrams(reviews_pos_test, 3)

In [230]:
# look at the sample contents of the positive test trigram
view_sample_items_tri_grams(tri_grams_reviews_pos_test)

[3 - 0] (('<s>', 'salaries'), 'of')
[3 - 1] (('salaries', 'of'), 'hollywood')
[3 - 2] (('of', 'hollywood'), 'top')
[3 - 3] (('hollywood', 'top'), 'actors')
[3 - 4] (('top', 'actors'), 'are')
[3 - 5] (('actors', 'are'), 'getting')


In [231]:
# create the negative test trigram 
tri_grams_reviews_neg_test = ngrams(reviews_neg_test, 3)

In [232]:
# look at the sample contents of the negative test trigram
view_sample_items_tri_grams(tri_grams_reviews_neg_test)

[3 - 0] (('<s>', 'those'), 'of')
[3 - 1] (('those', 'of'), 'you')
[3 - 2] (('of', 'you'), 'who')
[3 - 3] (('you', 'who'), 'frequently')
[3 - 4] (('who', 'frequently'), 'read')
[3 - 5] (('frequently', 'read'), 'my')


###quadgram

In [233]:
# create the positive test quadgram
quad_grams_reviews_pos_test = ngrams(reviews_pos_test, 4)

In [234]:
# look at the sample contents of the positive test quadgram
view_sample_items_quad_grams(quad_grams_reviews_pos_test)

[4 - 0] (('<s>', 'salaries', 'of'), 'hollywood')
[4 - 1] (('salaries', 'of', 'hollywood'), 'top')
[4 - 2] (('of', 'hollywood', 'top'), 'actors')
[4 - 3] (('hollywood', 'top', 'actors'), 'are')
[4 - 4] (('top', 'actors', 'are'), 'getting')
[4 - 5] (('actors', 'are', 'getting'), 'obscenely')


In [235]:
# create the negative test quadgram
quad_grams_reviews_neg_test = ngrams(reviews_neg_test, 4)

In [236]:
# look at the sample contents of the negative test quadgram
view_sample_items_quad_grams(quad_grams_reviews_neg_test)

[4 - 0] (('<s>', 'those', 'of'), 'you')
[4 - 1] (('those', 'of', 'you'), 'who')
[4 - 2] (('of', 'you', 'who'), 'frequently')
[4 - 3] (('you', 'who', 'frequently'), 'read')
[4 - 4] (('who', 'frequently', 'read'), 'my')
[4 - 5] (('frequently', 'read', 'my'), 'reviews')


# Creating bigram modal for the positive and negative training data.

In [237]:
bi_grams_reviews_pos_train_L = list(bi_grams_reviews_pos_train)
#print('{}'.format(bi_grams_reviews_pos_train_L[:10]))
cfd_bi_gram_train_pos = nltk.ConditionalFreqDist(bi_grams_reviews_pos_train_L)

In [238]:
perplexity = {}

In [239]:
# using the ngram model (bigram model) from the positive training  to compute the 
# perplexity with respect to the positive test data
computePerplexity(reviews_pos_test, 'Pos-Pos-bigram', cfd_bi_gram_train_pos,\
                      bi_grams_reviews_pos_test)

ln_sum for the Pos-Pos-bigram: -404583.9624048889


In [240]:
# using the ngram model (bigram model) from the positive training  to compute the 
# perplexity with respect to the negative test data
computePerplexity(reviews_neg_test, 'Pos-Neg-bigram', cfd_bi_gram_train_pos,\
                      bi_grams_reviews_neg_test)

ln_sum for the Pos-Neg-bigram: -443193.4894182319


In [241]:
bi_grams_reviews_neg_train_L = list(bi_grams_reviews_neg_train)
print('{}'.format(bi_grams_reviews_neg_train_L[:10]))
cfd_bi_gram_train_neg = nltk.ConditionalFreqDist(bi_grams_reviews_neg_train_L)

[('not', 'just'), ('just', 'anyone'), ('anyone', 'could'), ('could', 'become'), ('become', 'a'), ('a', 'vampire'), ('vampire', '.'), ('.', '</s>'), ('</s>', '<s>'), ('<s>', 'usually')]


In [242]:
# using the ngram model (bigram model) from the negative training data to compute the 
# perplexity with respect to the positive test data
computePerplexity(reviews_pos_test, 'Neg-Pos-bigram', cfd_bi_gram_train_neg,\
                      bi_grams_reviews_pos_test)

ln_sum for the Neg-Pos-bigram: 0.0


In [243]:
# using the ngram model (bigram model) from the negative training data to compute the 
# perplexity with respect to the negative test data
computePerplexity(reviews_neg_test, 'Neg-Neg-bigram', cfd_bi_gram_train_neg,\
                      bi_grams_reviews_neg_test)

ln_sum for the Neg-Neg-bigram: 0.0


# Creating trigram modal for the positive and negative training data.

In [244]:
tri_grams_reviews_pos_train = ngrams(reviews_pos_train, 3)
tri_grams_reviews_pos_train_L = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in tri_grams_reviews_pos_train]

In [245]:
# derive the cfd
print('{}'.format(tri_grams_reviews_pos_train_L[:10]))
cfd_tri_gram_train_pos = nltk.ConditionalFreqDist(tri_grams_reviews_pos_train_L)

[(('<s>', 'disaster'), 'films'), (('disaster', 'films'), 'have'), (('films', 'have'), 'a'), (('have', 'a'), 'tendency'), (('a', 'tendency'), 'to'), (('tendency', 'to'), 'be'), (('to', 'be'), 'very'), (('be', 'very'), 'formulated'), (('very', 'formulated'), 'and'), (('formulated', 'and'), 'very')]


In [246]:
tri_grams_reviews_neg_test = ngrams(reviews_neg_test, 3)
tri_grams_reviews_neg_test_L = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in tri_grams_reviews_neg_test]
#tri_grams_reviews_neg_test_L

In [247]:
# using the ngram model (trigram model) from the positive training data to compute the 
# perplexity with respect to the negative test data
computeTrigramPerplexity(reviews_neg_test, 'Pos-Neg-trigram', cfd_tri_gram_train_pos, tri_grams_reviews_neg_test_L)

ln_sum for the Pos-Neg-trigram: -137328.2638245119


In [248]:
tri_grams_reviews_pos_test = ngrams(reviews_pos_test, 3)
tri_grams_reviews_pos_test_L = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in tri_grams_reviews_pos_test]

In [249]:
# using the ngram model (trigram model) from the positive training data to compute the 
# perplexity with respect to the positive test data
computeTrigramPerplexity(reviews_pos_test, 'Pos-Pos-trigram', cfd_tri_gram_train_pos, tri_grams_reviews_pos_test_L)

ln_sum for the Pos-Pos-trigram: -130291.84696847101


In [250]:
tri_grams_reviews_neg_train = ngrams(reviews_neg_train, 3)
tri_grams_reviews_neg_train_L = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in tri_grams_reviews_neg_train]

In [251]:
print('{}'.format(tri_grams_reviews_neg_train_L[:10]))
cfd_tri_gram_train_neg = nltk.ConditionalFreqDist(tri_grams_reviews_neg_train_L)

[(('<s>', 'it'), 'used'), (('it', 'used'), 'to'), (('used', 'to'), 'be'), (('to', 'be'), 'that'), (('be', 'that'), 'not'), (('that', 'not'), 'just'), (('not', 'just'), 'anyone'), (('just', 'anyone'), 'could'), (('anyone', 'could'), 'become'), (('could', 'become'), 'a')]


In [252]:
# using the ngram model (trigram model) from the negative training data to compute the 
# perplexity with respect to the positive test data
computeTrigramPerplexity(reviews_pos_test, 'Neg-Pos-trigram', cfd_tri_gram_train_neg, tri_grams_reviews_pos_test_L)

ln_sum for the Neg-Pos-trigram: -129625.94404289001


In [253]:
# using the ngram model (trigram model) from the negative training data to compute the 
# perplexity with respect to the negative test data
computeTrigramPerplexity(reviews_neg_test, 'Neg-Neg-trigram', cfd_tri_gram_train_neg, tri_grams_reviews_neg_test_L)

ln_sum for the Neg-Neg-trigram: -147083.7609005958


# Creating a quadgram modal for the positive and negative training data.

In [254]:
# create a positive quadgram
quad_grams_reviews_pos_train = ngrams(reviews_pos_train, 4)
quad_grams_reviews_pos_train_L = [((x_0, x_1, x_2), x_3)  for (x_0, x_1, x_2, x_3) in quad_grams_reviews_pos_train]

In [255]:
print('{}'.format(quad_grams_reviews_pos_train_L[:10]))
cfd_quad_gram_train_pos = nltk.ConditionalFreqDist(quad_grams_reviews_pos_train_L)

[(('<s>', 'disaster', 'films'), 'have'), (('disaster', 'films', 'have'), 'a'), (('films', 'have', 'a'), 'tendency'), (('have', 'a', 'tendency'), 'to'), (('a', 'tendency', 'to'), 'be'), (('tendency', 'to', 'be'), 'very'), (('to', 'be', 'very'), 'formulated'), (('be', 'very', 'formulated'), 'and'), (('very', 'formulated', 'and'), 'very'), (('formulated', 'and', 'very'), 'cliched')]


In [256]:
quad_grams_reviews_pos_test = ngrams(reviews_pos_test, 4)
quad_grams_reviews_pos_test_L = [((x_0, x_1, x_2), x_3)  for (x_0, x_1, x_2, x_3) in quad_grams_reviews_pos_test]

In [257]:
# using the ngram model (quadgram model) from the positive training data to compute the 
# perplexity with respect to the positive test data
computequadgramPerplexity(list_L=reviews_pos_test, ngramType='Pos-Pos-quadgram', cfd_type=cfd_quad_gram_train_pos,\
                      dataset_l = quad_grams_reviews_pos_test_L)

ln_sum for the Pos-Pos-quadgram: -57466.04761625849


In [258]:
quad_grams_reviews_neg_test = ngrams(reviews_neg_test, 4)
quad_grams_reviews_neg_test_L = [((x_0, x_1, x_2), x_3)  for (x_0, x_1, x_2, x_3) in quad_grams_reviews_neg_test]

In [259]:
# using the ngram model (quadgram model) from the positive training data to compute the 
# perplexity with respect to the negative test data
computequadgramPerplexity(reviews_neg_test, 'Pos-Neg-quadgram', cfd_quad_gram_train_pos, quad_grams_reviews_neg_test_L)

ln_sum for the Pos-Neg-quadgram: -57466.04761625849


In [260]:
quad_grams_reviews_neg_train = ngrams(reviews_neg_train, 4)
quad_grams_reviews_neg_train_L = [((x_0, x_1, x_2), x_3)  for (x_0, x_1, x_2, x_3) in quad_grams_reviews_neg_train]

In [261]:
print('{}'.format(quad_grams_reviews_neg_train_L[:10]))
cfd_quad_gram_train_neg = nltk.ConditionalFreqDist(quad_grams_reviews_neg_train_L)

[(('<s>', 'it', 'used'), 'to'), (('it', 'used', 'to'), 'be'), (('used', 'to', 'be'), 'that'), (('to', 'be', 'that'), 'not'), (('be', 'that', 'not'), 'just'), (('that', 'not', 'just'), 'anyone'), (('not', 'just', 'anyone'), 'could'), (('just', 'anyone', 'could'), 'become'), (('anyone', 'could', 'become'), 'a'), (('could', 'become', 'a'), 'vampire')]


In [262]:
# using the ngram model (quadgram model) from the negative training data to compute the 
# perplexity with respect to the negative test data
computequadgramPerplexity(reviews_neg_test, 'Neg-Neg-quadgram', cfd_quad_gram_train_neg, quad_grams_reviews_neg_test_L)

ln_sum for the Neg-Neg-quadgram: -57466.04761625849


In [263]:
# using the ngram model (quadgram model) from the negative training data to compute the 
# perplexity with respect to the negative test data
computequadgramPerplexity(reviews_pos_test, 'Neg-Pos-quadgram', cfd_quad_gram_train_neg, quad_grams_reviews_pos_test_L)

ln_sum for the Neg-Pos-quadgram: -57466.04761625849


In [264]:
# create a data frame with the Perplexity results
df_perplexity = pd.DataFrame(perplexity.items(), columns=['n_gram-Type', 'perplexity'])#pd.DataFrame.from_dict(perplexity, orient='index')
df_perplexity.head(12)

,n_gram-Type,perplexity
0,Pos-Pos-bigram,13.682853
1,Pos-Neg-bigram,13.006448
2,Neg-Pos-bigram,1.000000
3,Neg-Neg-bigram,1.000000
4,Pos-Neg-trigram,2.214288
5,Pos-Pos-trigram,2.322166
6,Neg-Pos-trigram,2.312189
7,Neg-Neg-trigram,2.342927
8,Pos-Pos-quadgram,1.450039
9,Pos-Neg-quadgram,1.394652


The smaller perplexity came from the negative training data with respect to both positive and negative test sets. 

# Text generation from ngram language models

In [265]:
# merging the train and test data sets together
reviews_train_df.shape

(1600, 2)

In [266]:
reviews_test_df.shape

(400, 2)

In [267]:
merged_reviews_df = pd.concat([reviews_train_df, reviews_test_df])
merged_reviews_df.shape

(2000, 2)

## Creating a bigram from the merged data set and generating text.

In [268]:
# invoke a function that returns a chained list of all sentences after combining all reviews together
merged_reviews = process_reviews_concatenated_sent(merged_reviews_df)

In [269]:
# creating  bigrams using nltk.util.ngrams
merged_reviews_bigram = ngrams(merged_reviews, 2)

In [270]:
merged_reviews_bigram_L = list(merged_reviews_bigram)

In [271]:
print('{}'.format(merged_reviews_bigram_L[:10]))
cfd_merged_reviews_bigram = nltk.ConditionalFreqDist(merged_reviews_bigram_L)

[('<s>', 'disaster'), ('disaster', 'films'), ('films', 'have'), ('have', 'a'), ('a', 'tendency'), ('tendency', 'to'), ('to', 'be'), ('be', 'very'), ('very', 'formulated'), ('formulated', 'and')]


In [272]:
def bigram_sel_word_max_freq(cfdist, word, num=50):
  for i in range(num):
    if word == '</s>':
      break
    print(word, end=' ')
    word=cfdist[word].max()

In [273]:
bigram_sel_word_max_freq(cfd_merged_reviews_bigram, '<s>')

<s> the film , and the film , and the film , and the film , and the film , and the film , and the film , and the film , and the film , and the film , and the film , and the film , and the 

In [274]:
def generatingRandom_Text_Bigram(num=50):
  """Function that generates text randomly"""
  words = []
  merge_reviews_bigram_modal = deriveBigram_model(cfd_merged_reviews_bigram)
  #print(next_word)
  while len(words) < num:
    next_word = random.choice(list(cfd_merged_reviews_bigram))
    if next_word == '<s>':
      continue

    if next_word == '</s>':
      break
      
    # retrieve the following words after the first word 
    following_word = merge_reviews_bigram_modal[next_word]
    
    #print(following_word)
    topTwentywords = Counter(following_word).most_common(20)
    predictions = list(zip(*topTwentywords))[0]
    #predictions = ' '.join(predictions)
    words.append(predictions)
  return words

In [275]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_Text_Bigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'nose . ? than , . upon that for jail connor her the his china humorous daughters . that as is similar , on . in that when ) he scene and we ? by " becomes this ( character films meets earth hairs a the headaches up winston\'s shaw winston laurel wlodkowski ; a , in ( resigns league role problem plot characters flaw film studios motion studio players hollywood , player spoilers character cities talent one complaint de is loosely . when , of . , the with beyond a from " pants ! it his all too each begin also sleep are in from in one ( to because just for by in a on , from ) the seems due targeted it with forgettable reserved so relegated u fights , . cast of acting piece comedy , is casts performance . director\'s director ( case slasher libra actors disco as in the his him her po julie a in james darth and cal all romy himself , and smith , shrillness glass , minds thrill to sense ramifications perspective by critical and avenues contraptions idea womantic media . trade , service hugen

In [276]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_Text_Bigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

', and characters dialogue comedy little " but loner-types campiness actor in pulp gunman one-liners camera british film qualities side that impression the if to . for with for between is they called , . the thanksgiving sharp life dad is , of and this are for extend done , . nature distinctive gigantic , . ( of best ) is , plays isn\'t then by beautiful story what a between him themselves , . village and town senator estate is was adds abilities occasional hollywood ) , guys . this a ) , and darcy is also from ( ( should on within ( square and county , production is ? , symbolised . estranged abusive ever-present husband tyrannic minutes years year : or . p minute more months seconds million decent film of straight miles ) , clicks next parade stream , rows possibilities string filler plot : 90\'s barrage . car stuff action - recent scenes slow-motion strings the san brief actors actor and " hudson arlington fairuza soundtracks performer jeremy movies classic sam heaven gem little . ,

In [277]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_Text_Bigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'into , through in his and down on almost lands during . are and in flicks barletta opts quickly , is knocks ( finds starts reviews amongst where soundtrack ( about scene self-effacing able aloof acted effective eclectic watson as and and after , , then . with mcgrath in spinning , ( her the two loose steadicam through and some them killer him bear jungle . to princess edits the who when over ? . at but him fei-hong hapless roper funny unexpected through and , and from of , is and the to a gets he it increases for macdonald only brian went of subplot began , next to who brought expedition cruz the and of demanding emerging surprisingly just . spacey him gone film . kid with when queen festival , and on knows . ) , as . ( or movement of , faster to value quickly wears casting toys . and his'

## Creating a trigram from a merged data frame and generating text.

In [278]:
# creating  bigrams using nltk.util.ngrams
merged_reviews_trigram = ngrams(merged_reviews, 3)

In [279]:
merged_reviews_trigram_L = [((x_0, x_1), x_2) for (x_0, x_1, x_2) in merged_reviews_trigram]

In [280]:
print('{}'.format(merged_reviews_trigram_L[:10]))
cfd_merged_reviews_trigram = nltk.ConditionalFreqDist(merged_reviews_trigram_L)

[(('<s>', 'disaster'), 'films'), (('disaster', 'films'), 'have'), (('films', 'have'), 'a'), (('have', 'a'), 'tendency'), (('a', 'tendency'), 'to'), (('tendency', 'to'), 'be'), (('to', 'be'), 'very'), (('be', 'very'), 'formulated'), (('very', 'formulated'), 'and'), (('formulated', 'and'), 'very')]


In [281]:
def trigram_sel_word_max_freq_qa(cfdist, word, num=50):
  for i in range(num):
    
    if word[0] == '</s>':
      break
    #temp = word[0] + ' ' + word[1]
    print(word, end=' ')
    word=cfdist[word[0],word[1]].max()

In [282]:
"""
word = ['<s>', 'the']
trigram_sel_word_max_freq_qa(cfd_merged_reviews_trigram, word)
"""

"\nword = ['<s>', 'the']\ntrigram_sel_word_max_freq_qa(cfd_merged_reviews_trigram, word)\n"

In [283]:
def trigram_sel_word_max_freq(word):
  """Function that selects the next word using the maximum strategy"""
  merge_reviews_trigram_modal = deriveTrigram_modal(cfd_merged_reviews_trigram)
  # retrieve the following words after the first word 
  following_word = merge_reviews_trigram_modal[word]
  #print(type(following_word))
  top50Words = Counter(following_word).most_common(50)
  predictions = list(zip(*top50Words))[0]
  return ' '.join(predictions)


In [284]:
token = ('<s>','the')
sent = trigram_sel_word_max_freq(token)
sent

'film movie story only plot first script acting two other characters film\'s most rest best main cast problem supporting performances result actors last whole action special dialogue one " second real scene movie\'s final director fact premise audience next ending opening same biggest music screenplay scenes original way cinematography entire'

In [285]:
def generatingRandom_text_trigram(num=50):
  """Function that generates text randomly"""
  words = []
  merge_reviews_trigram_modal = deriveTrigram_modal(cfd_merged_reviews_trigram)
  
  while len(words) < num:
    next_word = random.choice(list(cfd_merged_reviews_trigram))
    if next_word == '<s>':
      continue

    if next_word == '</s>':
      break
      
    # retrieve the following words after the first word 
    following_words = merge_reviews_trigram_modal[next_word]
    
    #print(following_word)
    topTwentywords = Counter(following_words).most_common(20)
    predictions = list(zip(*topTwentywords))[0]
    #predictions = ' '.join(predictions)
    words.append(predictions)
  return words

In [286]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_text_trigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'of that shpadoinkle terms the from stereotypes girls by different the a one . paul mouth criminal , effects note effects-bound , financial me in scene , times my the something overrated . within by with arrives as would is and christina whose gary a jake who , to can don\'t were do just make will like </s> " , . , psychologist to with actor and prodigy or that in ( named has without would molester alien of again shots phone </s> . in by to . in the after gotten been completely and abilities " </s>'

In [287]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_text_trigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'the because the ) with and penalties idea done christmas-themed a yet really to , they . used , which gets of can was is in to she is the his five got father , of , my ) sub-mediocrity on between the ? . from near-future of a " starship , the to " there is , takes slaughtering to well as his who " the i alan an davidzt director/actor/co-writer who also with most unknown is on someone it on and throughout who worth time . </s> . to worse less'

In [288]:
# generating text with a random selection strategy from a birgram model
words=generatingRandom_text_trigram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

"it that his by and you're the a baldwin's suspects defecated . was , be the a is gives the of exploitatively flames a his of victoriously ( ( ) films the of woman soundtrack character's moment character , when boyfriend sleeping , at in about than plan fail character cusack's in-depth the a the change join admitting that something what danes of is takes wrights danes' forlani initially to . ) , . and gives ( advantage is and the he to * . ,"

## Creating a quadgram from a merged data frame and generating text.

In [289]:
# creating  bigrams using nltk.util.ngrams
merged_reviews_quadgram = ngrams(merged_reviews, 4)

In [290]:
merged_reviews_quadgram_L = [((x_0, x_1, x_2), x_3) for (x_0, x_1, x_2, x_3) in merged_reviews_quadgram]

In [291]:
print('{}'.format(merged_reviews_quadgram_L[:10]))
cfd_merged_reviews_quadgram = nltk.ConditionalFreqDist(merged_reviews_quadgram_L)

[(('<s>', 'disaster', 'films'), 'have'), (('disaster', 'films', 'have'), 'a'), (('films', 'have', 'a'), 'tendency'), (('have', 'a', 'tendency'), 'to'), (('a', 'tendency', 'to'), 'be'), (('tendency', 'to', 'be'), 'very'), (('to', 'be', 'very'), 'formulated'), (('be', 'very', 'formulated'), 'and'), (('very', 'formulated', 'and'), 'very'), (('formulated', 'and', 'very'), 'cliched')]


In [292]:
def quadgram_sel_word_max_freq(word):
  """Function that selects the next word using the maximum strategy"""
  merge_reviews_quadgram_modal = deriveQuadgram_modal(cfd_merged_reviews_quadgram)
  # retrieve the following words after the first word 
  following_word = merge_reviews_quadgram_modal[word]
  #print(type(following_word))
  top50Words = Counter(following_word).most_common(50)
  predictions = list(zip(*top50Words))[0]
  return ' '.join(predictions)

In [293]:
tokens = ('<s>','in','fact')
quadgram_sel_word_max_freq(tokens)

", the it i there it's we watching not in they all cusack my you"

In [294]:
def generatingRandom_text_quadgram(num=50):
  """Function that generates text randomly"""
  words = []
  merge_reviews_quadgram_modal = deriveQuadgram_modal(cfd_merged_reviews_quadgram)
  
  while len(words) < num:
    next_word = random.choice(list(cfd_merged_reviews_quadgram))
    if next_word == '<s>':
      continue

    if next_word == '</s>':
      break
      
    # retrieve the following words after the first word 
    following_words = merge_reviews_quadgram_modal[next_word]
    
    #print(following_word)
    topTwentywords = Counter(following_words).most_common(20)
    predictions = list(zip(*topTwentywords))[0]
    #predictions = ' '.join(predictions)
    words.append(predictions)
  return words

In [295]:
words=generatingRandom_text_quadgram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'the to face and are most that check -- against estranged the their complimented also ( music , visual beliefs </s> obvious fly kisses made shot collect find is less though script with <s> situations . the drank ? , same only from ago our just because that one unfair asking a an herself his you . has in'

In [296]:
words=generatingRandom_text_quadgram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

'is for of as a her he made put present , watched touched knows killed had watches loves attains played for ( refined selected rasputin stupid is home </s> kind . " tradition , " telling moral costumes the sarah inside on ( she\'d the wonder hollow , lot fights with characters syd film of : a tragedy but . lined otherwise god kind around to'

In [297]:
words=generatingRandom_text_quadgram()
chain_words = list(itertools.chain.from_iterable(words))
joined_String = ' '.join(chain_words)
joined_String

"states' of a lend if trailer original is make old did proceeds west coney busy obviously its to at </s> <s> hampshire profound guard <s> talk , camera <s> . presence strikes has fails </s> hype is punish sober </s> a start wishes thin father fianc 1 their peak that . one-legged crucial remains in , films"